In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split


In [34]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")

df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [35]:
df.shape

(4026, 7)

In [36]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [37]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [38]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [39]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [40]:
y=normalized_df["Period (P)"]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

In [42]:
from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplg

In [43]:
from sklearn.ensemble import GradientBoostingRegressor
from bayes_opt import BayesianOptimization

In [46]:
def bayesopt_objective(n_estimators,max_depth,min_samples_leaf,learning_rate):
    model= GradientBoostingRegressor(n_estimators=int(n_estimators)
              ,max_depth=int(max_depth)
              ,min_samples_leaf=int(min_samples_leaf)
              ,learning_rate=learning_rate
              )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [47]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
    
    
    opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    
    
    params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
  
    print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [48]:
def bayes_opt_validation(params_best):
    model= GradientBoostingRegressor(n_estimators=int(params_best['n_estimators'])
            ,max_depth=int(params_best['max_depth'])
            ,min_samples_leaf=int(params_best['min_samples_leaf'])
            ,learning_rate=params_best['learning_rate']
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [49]:
# Start time
start_time = time.time()

param_grid_simple={'n_estimators':(800,1500)
                  ,'max_depth':(3,10)
                  ,'min_samples_leaf':(2,20)
                   ,'learning_rate':( 0.001,0.5)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best 
validation_score=bayes_opt_validation(params_best) 
validation_score 
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   | learni... | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.004454  | 0.3365    | 6.417     | 16.86     | 822.0     |
| 2         | 0.004762  | 0.4042    | 6.959     | 7.357     | 832.7     |
| 3         | 0.006953  | 0.4953    | 3.048     | 15.86     | 1.323e+03 |
| 4         | 0.004016  | 0.1893    | 6.459     | 18.72     | 1.077e+03 |
| 5         | 0.00532   | 0.487     | 6.671     | 3.685     | 1.369e+03 |
| 6         | 0.00401   | 0.1066    | 6.88      | 7.261     | 1.371e+03 |
| 7         | 0.004434  | 0.4142    | 4.551     | 13.61     | 866.6     |
| 8         | 0.007507  | 0.2064    | 3.678     | 4.592     | 948.5     |
| 9         | 0.007721  | 0.2389    | 3.543     | 6.231     | 804.6     |
| 10        | 0.00507   | 0.4494    | 6.866     | 5.016     | 1.45e+03  |
| 11        | 0.007149  | 0.2715    | 3.292     | 11.45     | 1.248e+03 |
| 12        | 0.005766  | 0.4005    | 

| 67        | 0.007631  | 0.3623    | 3.405     | 4.738     | 806.8     |
| 68        | 0.007612  | 0.275     | 3.479     | 4.004     | 806.5     |
| 69        | 0.007687  | 0.2242    | 3.346     | 8.953     | 806.2     |
| 70        | 0.007541  | 0.3755    | 3.884     | 7.274     | 806.9     |
| 71        | 0.005978  | 0.2701    | 9.258     | 5.172     | 1.457e+03 |
| 72        | 0.008188  | 0.2883    | 9.966     | 2.835     | 803.5     |
| 73        | 0.004621  | 0.1685    | 4.054     | 9.105     | 807.2     |
| 74        | 0.007749  | 0.1785    | 3.234     | 7.209     | 806.7     |
| 75        | 0.00758   | 0.1537    | 3.329     | 13.61     | 1.142e+03 |
| 76        | 0.007767  | 0.1747    | 3.256     | 3.633     | 949.7     |
| 77        | 0.004638  | 0.0548    | 6.133     | 19.64     | 1.281e+03 |
| 78        | 0.007828  | 0.1175    | 3.96      | 4.188     | 949.5     |
| 79        | 0.0045    | 0.1465    | 9.671     | 11.82     | 989.7     |
| 80        | 0.008057  | 0.02778   | 

# GradientBoostingRegressor

In [51]:
from sklearn.ensemble import GradientBoostingRegressor
params_best['n_estimators'] = int(params_best['n_estimators'])
params_best['max_depth'] = int(params_best['max_depth'])
params_best['min_samples_leaf'] = int(params_best['min_samples_leaf'])
gbr_reg= GradientBoostingRegressor(**params_best )
gbr_reg.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.0025, max_depth=7, min_samples_leaf=6,
                          n_estimators=811)

In [52]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [53]:
y_pred= gbr_reg.predict(X_test)
y_pred1=gbr_reg.predict(X_val)
y_pred2=gbr_reg.predict(X_train)

In [54]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 0.02647873145991981
mean_squared_error: 0.0010872931337115165
rmse: 0.032974128247938814
r2 score: 0.9781925574465634


In [55]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 0.025491533027954757
mean_squared_error: 0.000959931849354261
rmse: 0.030982766973823708
r2 score: 0.9803756300787225


In [56]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 0.027189996254512388
mean_squared_error: 0.0011002266937276378
rmse: 0.03316966526402758
r2 score: 0.9785240592995932
